In [1]:
import yfinance as yf
import datetime
import ta
import datetime
import numpy as np
from ta import momentum, trend
from ta import momentum,trend

symbol = "^NSEI"
ticker = yf.Ticker(symbol)
end_date = datetime.date.today()
df = ticker.history(period="2d", interval="5m")

df['Date'] = df.index.date
df['Time'] = df.index.time


df['50EMA'] = trend.EMAIndicator(df['Close'], window=50).ema_indicator()
df['10EMA'] = trend.EMAIndicator(df['Close'], window=10).ema_indicator()


#  Trend Finder
df['SMA20'] =df['Close'].rolling(window=20).mean()
df['SMA40'] = df['Close'].rolling(window=40).mean()

range_threshold = 10
df['Trend'] = np.nan
df.loc[df['SMA20'] > df['SMA40'], 'Trend'] = 'Uptrend'

df.loc[df['SMA20'] < df['SMA40'], 'Trend'] = 'Downtrend'

df.loc[abs(df['SMA20'] - df['SMA40']) <= range_threshold, 'Trend'] = 'Sideways'


def Find_Up(row):
    if row['Close'] > row['Open']:
        return "Green"
    else:
        return "Red"

df['Candle'] = df.apply(Find_Up, axis=1)


df['Prev_Open'] = df['Open'].shift(1)
df['Prev_Close'] = df['Close'].shift(1)
df['Prev_High'] = df['High'].shift(1)
df['Prev_Low'] = df['Low'].shift(1)
df['Prev_Candle'] = df['Candle'].shift(1)
df['Prev_50EMA'] = df['50EMA'].shift(1)
df['Prev_10EMA'] = df['10EMA'].shift(1)


del df['Volume']
del df['Dividends']
del df['Stock Splits']
df = df.dropna()
# df.tail(25)

In [2]:
CE_HOLDING = False
ce_current_position = 0
ce_inisal_amount = 0

In [3]:
import pandas as pd
CE_data = pd.DataFrame(columns=['Entry Position', 'Entry DateTime', 'Exit DateTime', 'Current P&L', 'Exit Position','Trend'])

In [4]:
def find_10EMA_CE(row):
    global CE_HOLDING
    global ce_current_position
    global ce_inisal_amount
    global CE_data
    
    logic = row['Prev_Candle'] == "Red" and row['Candle'] == "Green" and row['Prev_High'] < row['Prev_10EMA']
    logic1 = row['Open'] - row['Low'] <= 1
    # print((row['Prev_Candle'] == "Green" and row['Candle'] == "Green"))
    if ((logic and logic1) or (row['Prev_Low'] > row['Prev_10EMA'] and row['Low'] > row['Prev_10EMA'] and row['Trend'] == "Uptrend") ) or (row['Prev_Candle'] == "Green" and row['Candle'] == "Green"):
        if not CE_HOLDING:
            ce_current_position = row['Close']
            CE_HOLDING = True
            entry_data = {
                'Entry Position': row['Close'],
                'Entry DateTime': datetime.datetime.combine(row['Date'], row['Time']),
                'Exit DateTime': None,
                'Current P&L': None,
                'Exit Position': None,
                'Trend': None
            }
            CE_data = pd.concat([CE_data, pd.DataFrame([entry_data])], ignore_index=True)
            return

    above = abs(row['10EMA'] - row['High'])
    total_range = abs(row['Low'] - row['High'])
    above_percentage = (above / total_range) * 100 if total_range != 0 else 0

    if  (row['High'] >= row['10EMA'] >= row['Low'] and above_percentage < 40 and row['Candle'] == "Red") or (row['10EMA'] > row['High'] and row['Candle'] == "Red") :
        if CE_HOLDING:
            current_pnl = (row['Close'] - ce_current_position)
            ce_inisal_amount += current_pnl
            CE_HOLDING = False
            CE_data.loc[CE_data.index[-1], 'Exit DateTime'] = datetime.datetime.combine(row['Date'], row['Time'])
            CE_data.loc[CE_data.index[-1], 'Current P&L'] = current_pnl
            CE_data.loc[CE_data.index[-1], 'Exit Position'] = row['Close']
            CE_data.loc[CE_data.index[-1], 'Trend'] = row['Trend']
            return

In [5]:
df.apply(find_10EMA_CE, axis=1)
from datetime import datetime
if CE_HOLDING:
    print("HOLDING")
    last_price_today = round(df.iloc[-1]['Close'],2)
    current_pnl  = last_price_today-ce_current_position
    ce_inisal_amount = ce_inisal_amount + current_pnl
    CE_data.loc[CE_data.index[-1], 'Exit DateTime'] =  f"{df.iloc[-1]['Date']} {df.iloc[-1]['Time']}"
    CE_data.loc[CE_data.index[-1], 'Current P&L'] = current_pnl
    CE_data.loc[CE_data.index[-1], 'Exit Position'] = last_price_today
    
print("TOTAL CE: ",ce_inisal_amount)

TOTAL CE:  -23.345703125


In [6]:
# sum(list(CE_data['Current P&L']))

In [7]:
# Convert Entry DateTime and Exit DateTime to datetime objects
CE_data['Entry DateTime'] = pd.to_datetime(CE_data['Entry DateTime'])
CE_data['Exit DateTime'] = pd.to_datetime(CE_data['Exit DateTime'])

# Calculate the time difference in minutes
CE_data['Tred Time'] = (CE_data['Exit DateTime'] - CE_data['Entry DateTime']).dt.total_seconds() / 60
# PE_data.to_csv("ENTRY EXIT.csv")
CE_data

,Entry Position,Entry DateTime,Exit DateTime,Current P&L,Exit Position,Trend,Tred Time
0,19104.250000,2023-10-31 13:40:00,2023-10-31 14:55:00,12.0,19116.25,Uptrend,75.0
1,19091.199219,2023-10-31 15:10:00,2023-10-31 15:15:00,-6.449219,19084.75,Sideways,5.0
2,19074.750000,2023-11-01 09:15:00,2023-11-01 09:25:00,-4.099609,19070.650391,Sideways,10.0
3,19044.099609,2023-11-01 10:10:00,2023-11-01 10:25:00,8.201172,19052.300781,Downtrend,15.0
4,19048.650391,2023-11-01 11:45:00,2023-11-01 12:35:00,-7.099609,19041.550781,Sideways,50.0
5,19033.449219,2023-11-01 13:40:00,2023-11-01 13:55:00,-16.548828,19016.900391,Sideways,15.0
6,19024.449219,2023-11-01 14:05:00,2023-11-01 14:10:00,-9.349609,19015.099609,Downtrend,5.0


In [8]:
sum(list(CE_data['Current P&L']))

-23.345703125

In [9]:
df

,Open,High,Low,Close,Date,Time,50EMA,10EMA,SMA20,SMA40,Trend,Candle,Prev_Open,Prev_Close,Prev_High,Prev_Low,Prev_Candle,Prev_50EMA,Prev_10EMA
Datetime,,,,,,,,,,,,,,,,,,,
2023-10-31 13:25:00+05:30,19089.949219,19118.900391,19083.849609,19118.900391,2023-10-31,13:25:00,19110.179185,19102.812379,19094.827734,19095.783887,Sideways,Green,19109.550781,19090.900391,19110.900391,19090.800781,Red,19109.823217,19099.237265
2023-10-31 13:30:00+05:30,19119.550781,19132.449219,19091.650391,19091.650391,2023-10-31,13:30:00,19109.452565,19100.782927,19095.755273,19095.491406,Sideways,Red,19089.949219,19118.900391,19118.900391,19083.849609,Green,19110.179185,19102.812379
2023-10-31 13:35:00+05:30,19092.150391,19099.199219,19088.900391,19099.199219,2023-10-31,13:35:00,19109.050473,19100.494980,19096.755273,19095.365137,Sideways,Green,19119.550781,19091.650391,19132.449219,19091.650391,Red,19109.452565,19100.782927
2023-10-31 13:40:00+05:30,19098.800781,19104.650391,19093.300781,19104.250000,2023-10-31,13:40:00,19108.862220,19101.177711,19097.655273,19095.483887,Sideways,Green,19092.150391,19099.199219,19099.199219,19088.900391,Green,19109.050473,19100.494980
2023-10-31 13:45:00+05:30,19104.599609,19113.800781,19101.150391,19105.000000,2023-10-31,13:45:00,19108.710760,19101.872672,19098.452734,19095.521387,Sideways,Green,19098.800781,19104.250000,19104.650391,19093.300781,Green,19108.862220,19101.177711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-01 14:30:00+05:30,19013.400391,19018.099609,18999.099609,19014.750000,2023-11-01,14:30:00,19036.083963,19017.470655,19018.504883,19032.056250,Downtrend,Green,19014.750000,19013.650391,19016.050781,19009.000000,Red,19036.954737,19018.075244
2023-11-01 14:35:00+05:30,19014.949219,19017.150391,19006.750000,19010.349609,2023-11-01,14:35:00,19035.074773,19016.175919,19017.124902,19031.238721,Downtrend,Red,19013.400391,19014.750000,19018.099609,18999.099609,Green,19036.083963,19017.470655
2023-11-01 14:40:00+05:30,19009.750000,19011.000000,19001.900391,19004.699219,2023-11-01,14:40:00,19033.883575,19014.089246,19015.642383,19030.223682,Downtrend,Red,19014.949219,19010.349609,19017.150391,19006.750000,Red,19035.074773,19016.175919
